In [2]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

floatx = "float32"
K.backend.set_floatx(floatx)

Using TensorFlow backend.


In [3]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [4]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [5]:
words, vocab = util.load_vocab("../vocab.txt", 10000)

In [6]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [7]:
#np.savez("embmat_100K.npz", emb_matrix=emb_matrix)

In [8]:
print(emb_matrix.shape, len(vocab))

(10001, 300) 10000


In [9]:
imp.reload(util)

seqlen = 128
stride = 128
X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride)
Yfake = np.zeros((Yu.shape[0], seqlen))

100%|##########| 22860/22860 [00:35<00:00, 652.49it/s]


In [10]:
Yfake = np.zeros((Yu.shape[0], seqlen))


In [11]:
emb_matrix.dtype

dtype('float32')

In [12]:
from keras.layers import Dense, Embedding, Reshape, Concatenate, CuDNNLSTM, Lambda
import keras as K
from keras import backend as B
import tensorflow as tf

In [13]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype=floatx, name="input_xu")
input_y = K.Input((seqlen,), dtype="int32", name="input_y")
input_yu = K.Input((seqlen,), dtype=floatx, name="input_yu")

reshaper_u  = K.layers.Reshape((seqlen,1))
resh_xu = reshaper_u(input_xu)
resh_yu = reshaper_u(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                          trainable=False,
                           weights=[emb_matrix],
                           name="embedding")


In [14]:
Lemb = L_emb
emb_x = Lemb(input_x)
emb_y = Lemb(input_y)

In [15]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
concat_y = Lconcat([emb_y, resh_yu])

In [16]:
def realdot(a,b):
    d = K.layers.multiply([a, b])
    return B.sum(d, axis=-1)

def dotlayer(x):
    a,b = x
    return realdot(a,b)

def cossim(x):
    a,b = x
    p = realdot(a,b)
    an = B.sqrt(realdot(a,a) + B.epsilon())
    bn = B.sqrt(realdot(b,b) + B.epsilon())
    return tf.divide(p, K.layers.multiply([an, bn]) + B.epsilon())

In [23]:
lstm =  K.layers.CuDNNGRU(256, return_sequences=True, name='gru1')(concat_x)
lstm = K.layers.BatchNormalization()(lstm)
#lstm =  K.layers.CuDNNLSTM(128, return_sequences=True, name='lstm2')(lstm)
lstm = K.layers.CuDNNGRU(256, return_sequences=True, name='gru2')(lstm)
lstm = K.layers.BatchNormalization()(lstm)
lstm = K.layers.CuDNNGRU(256, return_sequences=True, name='gru3')(lstm)
dense = K.layers.BatchNormalization()(lstm)

def candidate_gen(inp, layername):
    out = K.layers.Dense(dim, activation="relu", name=layername+"_1")(inp)
    out = K.layers.BatchNormalization()(out)
#    out = K.layers.Dense(dim, activation="relu", name=layername+"_2")(out)
#    out = K.layers.BatchNormalization()(out)
#    out = K.layers.Dense(dim, activation="relu", name=layername+"_3")(out)
#    out = K.layers.BatchNormalization()(out)
#    out = K.layers.Dense(dim, activation="relu", name=layername+"_4")(out)
#    out = K.layers.BatchNormalization()(out)
    out = K.layers.Dense(dim, activation="sigmoid", name=layername+"_last")(out)
    out = K.layers.BatchNormalization()(out)
    return out

out1 = candidate_gen(dense, "out1")
out2 = candidate_gen(dense, "out2")
out3 = candidate_gen(dense, "out3")

allout = K.layers.Concatenate(name="concat_out")([out1, out2, out3, dense])

#lstm3 =  K.layers.CuDNNLSTM(256, return_sequences=True, name='lstm3')(allout)

probs = K.layers.Dense(dim, activation="relu", name="probabilities_intermediate_1")(allout)
probs = K.layers.BatchNormalization()(probs)
#probs = K.layers.Dense(dim, activation="relu", name="probabilities_intermediate_2")(probs)
#probs = K.layers.BatchNormalization()(probs)
probs = K.layers.Dense(3, activation="softmax", name="probabilities")(probs)

def fexpander(x):
    return B.expand_dims(x)
L_expander = Lambda(fexpander, name="expander")

L_cossim = Lambda(cossim, name="cosine_similarity")
prox1 = L_cossim([concat_y, out1])
prox1 = L_expander(prox1)
prox2 = L_cossim([concat_y, out2])
prox2 = L_expander(prox2)
prox3 = L_cossim([concat_y, out3])
prox3 = L_expander(prox3)

allprox = Concatenate(name="concat_prox")([prox1, prox2, prox3])
final1 = K.layers.Lambda(dotlayer, name="final1")([probs, allprox])

#allones = B.ones(final1.shape, dtype="float32")
def final2_fun(x):
    return B.abs(1.0 - x)

final2 = Lambda(final2_fun, name="final2")(final1)
print(probs.shape)
print(prox3.shape)
print(allout.shape)
print(allprox.shape)
print(final1.shape)
print(final2.shape)




(?, 128, 3)
(?, 128, 1)
(?, 128, 1159)
(?, 128, 3)
(?, 128)
(?, 128)


In [24]:
model = K.Model(inputs=[input_x, input_xu, input_y, input_yu],
               outputs=[final2])

In [25]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_x (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_xu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 300)     3000300     input_x[0][0]                    
                                                                 input_y[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 128, 1)       0           input_xu[0][0]                   
          

In [26]:
opt = K.optimizers.Adam(lr=0.01)
model.compile(opt, loss='mean_squared_error')

In [27]:
1024*1.75

1792.0

In [28]:
model.fit([X, Xu, Y, Yu], [Yfake], batch_size=1024, epochs=1000000)


Epoch 1/1000000
 48128/126648 [==========>...................] - ETA: 51s - loss: 0.5162

KeyboardInterrupt: 

------------------------

In [67]:
predictor = K.Model([input_x, input_xu], [out1, out2, out3, probs])
predictor.compile(opt, loss="mean_absolute_error")

In [68]:
from sklearn import preprocessing as pp
norm_emb_matrix =  pp.normalize(Lemb.get_weights()[0], axis=1)

In [69]:
len(X)

300071

In [72]:
i = 10919
gen = X[i].tolist()
genu = Xu[i].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype=floatx)
results = []

def vec2word(x):
    we = np.transpose(pp.normalize(np.reshape(x[0, -1, :300], (1,300))))
    idx = np.argmax(np.matmul(norm_emb_matrix, we))
    return idx

iterations = 128

allcandidates = []
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], floatx)
    #print(tX)
    xv, xuv = 0, 0.0
    yhat = predictor.predict([tX, tXu])
    
    #for qq in yhat:
    #    print(qq.shape)
    #print(yhat)
    #break
    vecid = np.random.choice(3, p=yhat[3][0,0])
    z = yhat[vecid]
    
    
    candidates=[]
    for u,c in enumerate(yhat[:3]):
        cid = vec2word(c)
        candidates.append((words[cid], yhat[3][0,0,u]))
    allcandidates.append(candidates)
#    print(candidates)
    
    if False:# z[0, -1, -1] > 0.9:
        xuv = 1.0
        results.append("<UNK>")
    else:
        we = np.transpose(pp.normalize(np.reshape(z[0, -1, :300], (1,300))))
#        print(np.linalg.norm(we))
#        scores = np.matmul(norm_emb_matrix, we)
#        print(scores)
#        print(np.min(scores))
#        break
        xv = np.argmax(np.matmul(norm_emb_matrix, we))
        if xv:
            results.append(words[xv])
        else:
            results.append("<EMPTY>")
    gen.append(xv)
    genu.append(xuv)
    sys.stdout.write(results[-1]+" ")
    sys.stdout.flush()
    
    
for cc in allcandidates:
    print("| %20s %.4f | %20s %.4f | %20s %.4f |" %
         (cc[0][0], cc[0][1],
         cc[1][0], cc[1][1],
         cc[2][0], cc[2][1]))

exhibition , exhibition exhibition , exhibition exhibition , exhibition well - exhibition exhibitions , exhibitions , exhibitions , exhibitions , exhibitions , exhibitions , exhibitions , exhibitions , exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions , exhibitions , numerous exhibitions , well - been several exhibitions |                Wiels 0.0000 |           exhibition 1.0000 |                first 0.0000 |
|                    , 0.0000 |                    , 1.

In [51]:
print(np.min(z), np.max(z))

-3.9707959 11.670498


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'